In [1]:
#imports
#pip install contractions --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org
import gitlab
import mariadb
import sys
import datetime
import json
from dateutil import parser
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote

with open('Credentials.json') as f:
    data = json.load(f)
    mariadb_pwd = data['MariaDB_PWD']
    git_token = data['GIT_Token']


engine = create_engine('mysql+pymysql://root:%s@localhost/codereview' % quote(mariadb_pwd)) #creating instance of sql engine
gl = gitlab.Gitlab(url='GIT_DOMAIN_HERE', private_token=git_token) #creating gitlab instance
projectID = 3982 #5633 #8202 #3983 
project = gl.projects.get(projectID)

In [2]:
#store project data and retuns object of project
def store_project_data(projectID):
    project = gl.projects.get(projectID)
    pr_data={}
    pr_data['projectID'] = project.attributes['id']
    pr_data['name'] = project.attributes['name']
    pr_data['description'] = project.attributes['description']
    pr_data['webURL'] = project.attributes['web_url']
    try:
        conn = mariadb.connect(
            user="root",
            password="PWD_HERE",
            #host="192.0.2.1",
            port=3306,
            database="codereview"
        )
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        sys.exit(1)
    cur = conn.cursor()
    placeholder = ", ".join(["%s"] * len(pr_data))
    stmt = "insert into `{table}` ({columns}) values ({values});".format(table='project', columns=",".join(pr_data.keys()), values=placeholder)
    try: 
        cur.execute(stmt, list(pr_data.values()))
    except mariadb.Error as e: 
        print(f"Error: {e}")
    conn.commit() 
    #print(cur.lastrowid)
    conn.close()
    print(" COMPLETE")
    return project

In [ ]:
project = store_project_data(projectID)

In [3]:
#stores code review data for given project and time.
def store_code_review_data(project, createdAfter, createdBefore):
    mrs = project.mergerequests.list(state='merged', order_by='updated_at', created_after=createdAfter, created_before=createdBefore ,get_all=True)
    #insert merge request details
    merges = []
    mergeChanges = []
    commits = []
    notes = []
    diffs = []
    mr_commit = []
    for mr in mrs:
        try: 
            #         print('------- In MergeRequest ------')
            #         print(json.dumps(mr.attributes, indent=4))
            mr_data={}
            mr_commit_data = {}
            mr = project.mergerequests.get(mr.attributes['iid'])
            mr_data['MRIID'] = mr.attributes['iid']
            mr_data['projectID'] = projectID
            mr_data['title'] = mr.attributes['title']
            mr_data['description'] = mr.attributes['description']
            mr_data['state'] = mr.attributes['state']
            mr_data['createdAt'] = parser.parse(mr.attributes['created_at']).strftime('%Y-%m-%d %H:%M:%S')
            mr_data['mergedAt'] = parser.parse(mr.attributes['merged_at']).strftime('%Y-%m-%d %H:%M:%S')
            mr_data['targetBranch'] = mr.attributes['target_branch']
            mr_data['sourceBranch'] = mr.attributes['source_branch']
            mr_data['type'] = mr.attributes['source_branch'].split('/')[0]
            mr_data['userNotesCount'] = mr.attributes['user_notes_count']
            mr_data['reviewerCount'] = str(len(mr.attributes['reviewers']))
            mr_data['changesCount'] = mr.attributes['changes_count']
            mr_data['webURL'] = mr.attributes['web_url']
            mr_data['totalLinesAdded'] = 0
            mr_data['totalLinesRemoved'] = 0
            merges.append(mr_data)
        except mariadb.Error as e: 
            print('------- In MergeRequest ------')
            print(f"Error: {e}")
    
        for change in mr.changes(get_all=True)['changes']:
            try: 
                #             print('------- In MergeRequestChange ------')
                #             print(json.dumps(change, indent=4))
                change_data = {}
                change_data['MRIID'] = mr.attributes['iid']
                change_data['oldPath'] = change['old_path']
                change_data['newPath'] = change['new_path']
                change_data['newFiles'] = change['new_file']
                change_data['renamedFiles'] = change['renamed_file']
                change_data['deletedFiles'] = change['deleted_file']
                change_data['diff'] = change['diff']
                change_data['linesAdded'] = change['diff'].count("\n+")
                change_data['linesRemoved'] = change['diff'].count("\n-")
                mr_data['totalLinesAdded'] = mr_data['totalLinesAdded'] + change_data['linesAdded']
                mr_data['totalLinesRemoved'] = mr_data['totalLinesRemoved'] + change_data['linesRemoved']
                mergeChanges.append(change_data)
            except mariadb.Error as e: 
                print('------- In MergeRequestChange ------')
                print(f"Error: {e}")
    
        for note in mr.notes.list(order_by="updated_at", sort="asc", get_all=True):
            try:
                #             print('------- In Notes ------')
                #             print(json.dumps(note.attributes, indent=4))
                note_data = {}
                note_data['noteID'] = note.id
                note_data['MRIID'] = note.mr_iid
                note_data['type'] = note.type
                note_data['body'] = note.body
                note_data['createdAt'] = parser.parse(note.created_at).strftime('%Y-%m-%d %H:%M:%S')
                note_data['updatedAt'] = parser.parse(note.updated_at).strftime('%Y-%m-%d %H:%M:%S')
                note_data['system'] = note.system
                notes.append(note_data)
            except mariadb.Error as e: 
                print('------- In Notes ------')
                print(f"Error: {e}")

        for commit in mr.commits(order_by="updated_at", get_all=True):
            try: 
                #             print('------- In Commit-------')
                #             print(json.dumps(commit.attributes, indent=4))
                c_data={}
                c_data['commitID'] = commit.attributes['id']
                c_data['shortID'] = commit.attributes['short_id']
#                 c_data['MRIID'] = mr_data['MRIID']
                c_data['title'] = commit.attributes['title']
                c_data['message'] = commit.attributes['message']
                c_data['committedDate'] = parser.parse(commit.attributes['committed_date']).strftime('%Y-%m-%d %H:%M:%S')
                c_data['webURL'] = commit.attributes['web_url']
                c_data['totalLinesAdded'] = 0
                c_data['totalLinesRemoved'] = 0
                commits.append(c_data)
                mr_commit_data['MRIID'] = mr.attributes['iid']
                mr_commit_data['commitID'] = commit.attributes['id']
                mr_commit.append(mr_commit_data)
            except mariadb.Error as e:
                print('------- In Commit-------')
                print(f"Error: {e}")
            for d in commit.diff(get_all=True):
                try:
                    #                 print('------- In diff ------')
                    #                 print(json.dumps(d,indent=4))
                    diff_data={}
                    diff_data['commitID'] = commit.attributes['id']
                    diff_data['oldPath'] = d['old_path']
                    diff_data['newPath'] = d['new_path']
                    diff_data['newFiles'] = d['new_file']
                    diff_data['renamedFiles'] = d['renamed_file']
                    diff_data['deletedFiles'] = d['deleted_file']
                    diff_data['diff'] = d['diff']
                    diff_data['linesAdded'] = d['diff'].count("\n+")
                    diff_data['linesRemoved'] = d['diff'].count("\n-")
                    c_data['totalLinesAdded'] = c_data['totalLinesAdded'] + diff_data['linesAdded']
                    c_data['totalLinesRemoved'] = c_data['totalLinesRemoved'] + diff_data['linesRemoved']
                    diffs.append(diff_data)
                except mariadb.Error as e:
                    print('------- In diff ------')
                    print(f"Error: {e}")
    #creating pandas dataFrames of all the collected data
    mergeRequestDF = pd.json_normalize(merges)
    mergeChangesDF = pd.json_normalize(mergeChanges)
    commitsDF = pd.json_normalize(commits)
    notesDF = pd.json_normalize(notes)
    diifsDF = pd.json_normalize(diffs)
    mrCommitDF = pd.json_normalize(mr_commit)
    #removing duplicates
    mergeRequestDF = mergeRequestDF.drop_duplicates()
    mergeChangesDF = mergeChangesDF.drop_duplicates()
    commitsDF = commitsDF.drop_duplicates()
    notesDF = notesDF.drop_duplicates()
    diifsDF = diifsDF.drop_duplicates()
    mrCommitDF = mrCommitDF.drop_duplicates()
    #storing dataFrames into sql
    mergeRequestDF.to_sql('mergerequest', con = engine, if_exists='append', index= False)
    mergeChangesDF.to_sql('mrchange', con = engine, if_exists='append', index= False)
    commitsDF.to_sql('commits', con = engine, if_exists='append', index= False)
    notesDF.to_sql('notes', con = engine, if_exists='append', index= False)
    diifsDF.to_sql('diff', con = engine, if_exists='append', index= False)
    mrCommitDF.to_sql('mr_commits', con = engine, if_exists='append', index= False)
    print('Data stored for project '+ str(project.id))
    return mergeRequestDF, mergeChangesDF, commitsDF, mrCommitDF, notesDF, diifsDF

In [6]:
mergeRequestDF, mergeChangesDF, commitsDF,mrCommitDF, notesDF, diifsDF = store_code_review_data(project, createdAfter='2022-07-01T00:00:00Z',createdBefore='2022-10-01T00:00:00Z')

Data stored for project 3982


In [4]:
mergeRequestDF, mergeChangesDF, commitsDF,mrCommitDF, notesDF, diifsDF = store_code_review_data(project, createdAfter='2022-04-01T00:00:00Z',createdBefore='2022-07-01T00:00:00Z')

Data stored for project 3982


In [5]:
mergeRequestDF

,MRIID,projectID,title,description,state,createdAt,mergedAt,targetBranch,sourceBranch,type,userNotesCount,reviewerCount,changesCount,webURL,totalLinesAdded,totalLinesRemoved
0,1834,3982,Feature/2022/s19/update fr default landing bac...,,merged,2022-06-24 08:53:45,2022-08-17 11:40:57,develop,feature/2022/s19/update_fr_default_landing_bac...,feature,0,0,5,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,0,4
1,1744,3982,[CDTE-26721] Revive sonarqube,Related to CDTE-26721,merged,2022-05-19 09:25:22,2022-07-15 12:00:53,develop,feature/2022/S17/CDTE-26721/revive_sonarqube,feature,2,0,12,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,179,157
2,1836,3982,[CDTE-27849] Updated search params from SPECIF...,Related to CDTE-27849,merged,2022-06-24 11:46:59,2022-07-12 10:38:15,develop,feature/2022/S19/CDTE-27849_searchRanking,feature,0,0,2,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,2,2
3,1851,3982,eprescription-rewrite,,merged,2022-06-30 10:54:24,2022-07-05 10:33:46,develop,feature/eprescription-rewrite,feature,0,0,498,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,633,798
4,1849,3982,CDTE-31019 - Update Prescription data layer: R...,- refactor on prescription data source - remov...,merged,2022-06-29 12:00:27,2022-07-05 09:42:12,feature/eprescription-rewrite,feature/CDTE-30046-Refactor/CDTE-31019,feature,6,0,9,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,70,328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,1619,3982,CDTE-26347,- added RV scroll helper\n- check if there is ...,merged,2022-04-01 12:20:38,2022-04-01 13:29:07,feature/booking-assistant-v2,feature/CDTE-26347-weekly-calendar-view-animat...,feature,7,0,6,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,62,18
231,1618,3982,CDTE-26343,Added Segur implemetation (Document Listing & ...,merged,2022-04-01 12:19:02,2022-04-01 12:30:06,feature/segur,feature/CDTE-26343_documents_section,feature,1,0,2,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,2,3
232,1617,3982,CDTE-26343,Added Segur implemetation (Document Listing & ...,merged,2022-04-01 11:32:32,2022-04-01 11:35:02,feature/segur,feature/CDTE-26343_documents_section,feature,1,0,5,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,22,9
233,1615,3982,[CDTE-25704] Remove extra grey line from searc...,,merged,2022-04-01 10:50:26,2022-04-01 10:55:24,release/22.2.0,fix/2022/S13/CDTE-25704/remove_extra_grey_line,fix,0,0,1,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,1,1


In [10]:
notesDF[notesDF['createdAt']>'2022-07-01 00:00:00']

,noteID,MRIID,type,body,createdAt,updatedAt,system
5,2120951,1834,None,added 227 commits\n\n<ul><li>ee0b16fa...d705e7...,2022-08-17 11:40:44,2022-08-17 11:40:44,True
14,2028815,1744,DiffNote,changed this line in [version 5 of the diff](/...,2022-07-04 12:51:30,2022-07-04 12:51:30,True
15,2028816,1744,None,added 1 commit\n\n<ul><li>1c7907f3 - CDTE-2672...,2022-07-04 12:51:31,2022-07-04 12:51:31,True
16,2028956,1744,DiffNote,changed this line in [version 6 of the diff](/...,2022-07-04 13:23:04,2022-07-04 13:23:04,True
17,2028957,1744,None,added 1 commit\n\n<ul><li>a1451741 - CDTE-2672...,2022-07-04 13:23:05,2022-07-04 13:23:05,True
...,...,...,...,...,...,...,...
113,2024521,1853,None,added 1 commit\n\n<ul><li>5c6b271a - booking-a...,2022-07-01 06:38:08,2022-07-01 06:38:08,True
114,2024523,1853,DiffNote,"fixed, seems the method was not even used",2022-07-01 06:38:27,2022-07-01 06:38:27,False
117,2024708,1853,DiffNote,Fields are nullable because the nullability co...,2022-07-01 07:22:48,2022-07-01 07:22:48,False
118,2024748,1853,None,resolved all threads,2022-07-01 07:38:19,2022-07-01 07:38:20,True


In [13]:
mergeRequestDF[mergeRequestDF['createdAt']<='2022-07-01 00:00:00']

,MRIID,projectID,title,description,state,createdAt,mergedAt,targetBranch,sourceBranch,type,userNotesCount,reviewerCount,changesCount,webURL,totalLinesAdded,totalLinesRemoved
0,1834,3982,Feature/2022/s19/update fr default landing bac...,,merged,2022-06-24 08:53:45,2022-08-17 11:40:57,develop,feature/2022/s19/update_fr_default_landing_bac...,feature,0,0,5,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,0,4
1,1744,3982,[CDTE-26721] Revive sonarqube,Related to CDTE-26721,merged,2022-05-19 09:25:22,2022-07-15 12:00:53,develop,feature/2022/S17/CDTE-26721/revive_sonarqube,feature,2,0,12,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,179,157
2,1836,3982,[CDTE-27849] Updated search params from SPECIF...,Related to CDTE-27849,merged,2022-06-24 11:46:59,2022-07-12 10:38:15,develop,feature/2022/S19/CDTE-27849_searchRanking,feature,0,0,2,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,2,2
3,1851,3982,eprescription-rewrite,,merged,2022-06-30 10:54:24,2022-07-05 10:33:46,develop,feature/eprescription-rewrite,feature,0,0,498,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,633,798
4,1849,3982,CDTE-31019 - Update Prescription data layer: R...,- refactor on prescription data source - remov...,merged,2022-06-29 12:00:27,2022-07-05 09:42:12,feature/eprescription-rewrite,feature/CDTE-30046-Refactor/CDTE-31019,feature,6,0,9,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,70,328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,1619,3982,CDTE-26347,- added RV scroll helper\n- check if there is ...,merged,2022-04-01 12:20:38,2022-04-01 13:29:07,feature/booking-assistant-v2,feature/CDTE-26347-weekly-calendar-view-animat...,feature,7,0,6,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,62,18
231,1618,3982,CDTE-26343,Added Segur implemetation (Document Listing & ...,merged,2022-04-01 12:19:02,2022-04-01 12:30:06,feature/segur,feature/CDTE-26343_documents_section,feature,1,0,2,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,2,3
232,1617,3982,CDTE-26343,Added Segur implemetation (Document Listing & ...,merged,2022-04-01 11:32:32,2022-04-01 11:35:02,feature/segur,feature/CDTE-26343_documents_section,feature,1,0,5,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,22,9
233,1615,3982,[CDTE-25704] Remove extra grey line from searc...,,merged,2022-04-01 10:50:26,2022-04-01 10:55:24,release/22.2.0,fix/2022/S13/CDTE-25704/remove_extra_grey_line,fix,0,0,1,https://git.cgm.ag/cgm-de-life-apps/cgm-clickd...,1,1


In [14]:
notesDF

,noteID,MRIID,type,body,createdAt,updatedAt,system
0,2012110,1834,None,approved this merge request,2022-06-24 09:14:37,2022-06-24 09:14:37,True
1,2012141,1834,None,added 1 commit\n\n<ul><li>ff129d8a - [CDTE-297...,2022-06-24 09:19:01,2022-06-24 09:19:01,True
2,2012150,1834,None,added 4 commits\n\n<ul><li>ff129d8a...0fc6a8ac...,2022-06-24 09:20:36,2022-06-24 09:20:36,True
3,2012158,1834,None,approved this merge request,2022-06-24 09:23:14,2022-06-24 09:23:14,True
4,2019415,1834,None,added 25 commits\n\n<ul><li>b0c28e87...08b02ef...,2022-06-29 07:08:12,2022-06-29 07:08:12,True
...,...,...,...,...,...,...,...
1927,1836879,1617,None,ok,2022-04-01 11:34:49,2022-04-01 11:34:49,False
1928,1836721,1615,None,added 2 commits\n\n<ul><li>f5711c62 - 1 commit...,2022-04-01 10:52:10,2022-04-01 10:52:10,True
1929,1836728,1615,None,approved this merge request,2022-04-01 10:54:40,2022-04-01 10:54:40,True
1930,1835877,1614,None,assigned to @doru.nechifor,2022-04-01 07:59:48,2022-04-01 07:59:48,True
